In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
######################################################################## 100.0%#=#=#                                                                         
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
from langchain_community.llms import Ollama
from IPython.display import Markdown
from pathlib import Path

In [4]:
from crewai import Agent, Task, Crew
from crewai_tools import BaseTool

class PromptingPatientTool(BaseTool):
    name: str = "Patient Prompting"
    description: str = "When more information is needed from the patient, use this tool. "
    "Don't comment on the result afterwards. Do not add anything. Just return the result. "

    def _run(self, question: str) -> str:
        # Your tool's logic here
        return f"{question}"
    


/root/.local/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


### Profile Tool

In [7]:
import json
from typing import Dict, Any, Optional

# Assuming BaseTool definition is available
class ProfileTool(BaseTool):
    name: str = "Patient Profile"
    description: str = "Handles saving and updating patient profiles in JSON format."

    def _run(self, operation: str, patient_id: Optional[str] = None, updated_info: Optional[Dict[str, Any]] = None, file_path: Optional[str] = "patient_profile.json") -> str:
        if not patient_id or not updated_info:
            return "Invalid operation or missing parameters."

        # Load existing profile if it exists
        try:
            with open(file_path, 'r') as f:
                profile = json.load(f)
        except FileNotFoundError:
            profile = {}
            
        # Ensure the profile includes all relevant fields
        required_fields = {
            "patient_id": "",
            "symptoms": "",
            "medical_history": {
                "current_medications": [],
                "lifestyle_factors": {},
                "allergies": [],
                "recent_lab_results": []
            },
            "diagnoses": [],
            "treatment_plan": {},
            "trigger": {}
        }

        # Merge required fields with the provided updated_info
        updated_info = {**required_fields, **updated_info}

        if operation == "save":
            # Save or overwrite the profile information
            profile[patient_id] = updated_info
            message = f"Profile for patient {patient_id} saved successfully."

        elif operation == "update":
            # Update the existing profile or create a new one if it doesn't exist
            if patient_id in profile:
                profile[patient_id].update(updated_info)
                message = f"Profile for patient {patient_id} updated successfully."
            else:
                profile[patient_id] = updated_info
                message = f"Profile for patient {patient_id} did not exist and has been created."

        else:
            return "Invalid operation."

        # Write the profile back to the JSON file
        try:
            with open(file_path, 'w') as f:
                json.dump(profile, f, indent=4)
        except IOError as e:
            return f"Failed to save the profile: {str(e)}"

        return message

# Example usage:
profile_tool = ProfileTool()

In [12]:
from crewai import Agent, Task, Crew, Process
from agents import get_med_crew

llm = Ollama(model='mistral-nemo')
bio_llm = Ollama(model='jsk/bio-mistral')

nurse = Agent(
    role="A hospital nurse",
    goal="Your goal is to gather patient information and symptoms by coming up with a single question for the patient to obtain pertinent information."
    "You can only output either a question or a summary of the information returned by your tools",
    backstory="You are modeled after a seasoned nurse with extensive experience in primary care."
    "Been trained on a wide range of medical conditions and symptoms,"
    "allowing you to ask pertinent questions and recognize key details."
    "You effectively elicit and document a comprehensive list of symptoms gathered from patients, ensuring the information is precise and thorough."
    "This will help other agents in diagnosing and treating the patient more efficiently."
    "Your persona is designed to be empathetic and attentive, reflecting a caring and professional demeanor."
    "you are equipped with a robust knowledge base to handle diverse medical queries"
    "and provide a supportive conversational experience.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [14]:
keyword = "True"   
collect_info = Task(
    description=("Given this chat history between you and the patient: \"{history}\" ."
                 "Highlight important symptoms. "
                 "You will collect patient information by asking questions. You will output a question for further precision. "
                 "If enough information has been recieved, report the information in a bullet point format to the doctor. "
                "Avoid adding unnecessary syntax like quotation and asterisks marks. "
                "Ensure to never repeat questions. "),
    expected_output=f"One question about the patient's symptoms. In addition, simply add {keyword} if it is a question for the patient exclusively at the end. Either that, or a bullet list of all of the patient's symptoms. " ,
    tools=[profile_tool],

    agent=nurse
)

In [15]:
prompt_crew = Crew(
    agents=[nurse],
    tasks=[collect_info],
    verbose=True,
)


In [16]:
diagnose_crew = get_med_crew()

2024-08-24 13:10:27,066 - 140719035224064 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
diagnose_crew.kickoff(inputs={'symptoms':'pain in lower back, 10/10 severity'})

In [21]:
import gradio as gr
from gradio import ChatMessage

In [22]:
import random
import torch

def filter_ans(ans):
    return ans.removeprefix(keyword).removesuffix(keyword).strip("\"* ")
    
def response(message, history):
    q_and_ans = []
    history = history[-10:]
    history += [[message, None]]
    for i in range(len(history)-1):
        q_and_ans.append(' '.join([history[i][1], history[i+1][0]]))
    
    history_str = '\n'.join(q_and_ans)
    if history_str == "": history_str = "None yet"

    answer = prompt_crew.kickoff(inputs={'history': history_str}).raw
    if keyword not in answer:
        return filter_ans(diagnose_crew.kickoff(inputs={'symptoms': answer}).raw)
    return filter_ans(answer)

def init_convo():
    return filter_ans(prompt_crew.kickoff(inputs={'history': "None yet"}).raw)

gr.ChatInterface(
    response,
    chatbot=gr.Chatbot([(None, init_convo())])
).launch(share=True)

 [2024-08-24 13:15:20][DEBUG]: == Working Agent: A hospital nurse
 [2024-08-24 13:15:20][INFO]: == Starting Task: Given this chat history between you and the patient: "None yet" .Highlight important symptoms. You will collect patient information by asking questions. You will output a question for further precision. If enough information has been recieved, report the information in a bullet point format to the doctor. Avoid adding unnecessary syntax like quotation and asterisks marks. Ensure to never repeat questions. 


> Entering new CrewAgentExecutor chain...
Thought: I should start by asking about the patient's primary reason for visiting.
Action: Patient Profile
Action Input: {"operation": "load", "patient_id": None 

Invalid operation or missing parameters.

Thought:
Thought: The tool was used incorrectly. I should start by asking about the patient's primary reason for visiting.
Action: Patient Profile
Action Input: {"operation": "load", "patient_id": None 

I tried reusing the sa

 [2024-08-24 13:16:01][DEBUG]: == Working Agent: A hospital nurse
 [2024-08-24 13:16:01][INFO]: == Starting Task: Given this chat history between you and the patient: "Can you tell me what brings you to the hospital today? i have a cut on my hand" .Highlight important symptoms. You will collect patient information by asking questions. You will output a question for further precision. If enough information has been recieved, report the information in a bullet point format to the doctor. Avoid adding unnecessary syntax like quotation and asterisks marks. Ensure to never repeat questions. 


> Entering new CrewAgentExecutor chain...
Action: Patient Profile
Action Input: {"operation": "read", "patient_id": "1 

Invalid operation or missing parameters.

Thought:
Action: Patient Profile
Action Input: {"operation": "create", "patient_id": "1", "updated_info": {"reason_for_visit": "Cut on hand" 

Invalid operation.

Thought:
Action: Patient Profile
Action Input: {"operation": "create", "patien

In [36]:
keyword in "hi"

False

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte